In [24]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [25]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Use CUDA</h2>

In [26]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
device   = torch.device("cuda" if use_cuda else "cpu")

True


<h2>Create Environments</h2>

In [27]:
from common.multiprocessing_env import SubprocVecEnv

num_envs = 3
#env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"

# def make_env():
#     def _thunk():
#         env = gym.make(env_name)
#         return env

#     return _thunk

# envs = [make_env() for i in range(num_envs)]
# envs = SubprocVecEnv(envs)

env = gym.make(env_name)

[2018-06-11 20:56:21,991] Making new env: FrozenLake-v0


<h2>Neural Network</h2>

In [28]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)


class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [29]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

<h1>High-Dimensional Continuous Control Using Generalized Advantage Estimation</h1>
<h3><a href="https://arxiv.org/abs/1506.02438">Arxiv</a></h3>

In [30]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [31]:
print(env.observation_space.n)
print(env.action_space.n)

num_inputs  = env.observation_space.n
num_outputs = 1#env.action_space.n

#Hyper params:
hidden_size = 10 #256
lr          = 3e-2
num_steps   = 20

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters())


16
4


In [32]:
max_frames   = 100000
frame_idx    = 0
test_rewards = []

In [33]:
def state_fix(evir,states):
    new_state = np.zeros(evir.observation_space.n)
    new_state[states] =  1
    return new_state

In [34]:
def action_fix(envir,actions):
    new_action = np.zeros((len(actions),1)
    for coutner,action in enumerate(actions):
        pass #new_action[counter] = np.argmax

SyntaxError: invalid syntax (<ipython-input-34-836166f5e286>, line 3)

In [38]:
state = env.reset()
state = state_fix(env,state)
print(state)

while frame_idx < max_frames:

    log_probs = []
    values    = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        print("start")
        state = torch.FloatTensor(state).to(device).unsqueeze(0)
        dist, value = model(state)

        action = dist.sample()
        print(action.cpu().numpy()[0])
  

        next_state, reward, done, _ = env.step(0)
        print(next_state)
        print(reward)
        
        # Log probablity of action
        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        #List of rewards
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        #print(rewards)
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_rewards.append(np.mean([test_env() for _ in range(10)]))
            plot(frame_idx, test_rewards)
            
    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    
    returns = compute_gae(next_value, rewards, masks, values)
    log_probs = torch.cat(log_probs)
    returns   = torch.cat(returns).detach()
    values    = torch.cat(values)

    advantage = returns - values

    actor_loss  = -(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()

    loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
start
[-0.34478518]
4
0.0


TypeError: new(): data must be a sequence (got float)

In [ ]:
test_env(True)